In [33]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import folium
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from shapely.geometry import Point, Polygon
from shapely.ops import nearest_points

In [2]:
# Workaround to fix chrome issue where folium won't plot maps with a large number of layers
# See comment by dstein64 at: https://github.com/python-visualization/folium/issues/812

import base64
def _repr_html_(self, **kwargs):
    html = base64.b64encode(self.render(**kwargs).encode('utf8')).decode('utf8')
    onload = (
        'this.contentDocument.open();'
        'this.contentDocument.write(atob(this.getAttribute(\'data-html\')));'
        'this.contentDocument.close();'
    )
    if self.height is None:
        iframe = (
            '<div style="width:{width};">'
            '<div style="position:relative;width:100%;height:0;padding-bottom:{ratio};">'
            '<iframe src="about:blank" style="position:absolute;width:100%;height:100%;left:0;top:0;'
            'border:none !important;" '
            'data-html={html} onload="{onload}" '
            'allowfullscreen webkitallowfullscreen mozallowfullscreen>'
            '</iframe>'
            '</div></div>').format
        iframe = iframe(html=html, onload=onload, width=self.width, ratio=self.ratio)
    else:
        iframe = ('<iframe src="about:blank" width="{width}" height="{height}"'
                  'style="border:none !important;" '
                  'data-html={html} onload="{onload}" '
                  '"allowfullscreen" "webkitallowfullscreen" "mozallowfullscreen">'
                  '</iframe>').format
        iframe = iframe(html=html, onload=onload, width=self.width, height=self.height)
    return iframe

folium.branca.element.Figure._repr_html_ = _repr_html_


In [3]:
def gridify_polygon(poly,grid_spacing):
    # poly: polygon which we want a grid inside
    # grid_spacing: spaceing in lattitude/longitude degrees
    poly_xmin,poly_ymin,poly_xmax,poly_ymax = poly.geometry.total_bounds

    cols = list(np.arange(poly_xmin,poly_xmax+grid_spacing,grid_spacing))
    rows = list(np.arange(poly_ymin,poly_ymax+grid_spacing,grid_spacing))
    rows.reverse()

    polygons = []
    for x in cols:
        for y in rows:
            polygons.append( Polygon([(x,y), (x+grid_spacing, y), (x+grid_spacing, y-grid_spacing), (x, y-grid_spacing)]) )

    grid = gpd.GeoDataFrame({'geometry':polygons})
    grid['center_x'] = grid['geometry'].centroid.x
    grid['center_y'] = grid['geometry'].centroid.y
    grid['center'] = grid['geometry'].centroid

    grid['isin_poly'] = grid.apply(lambda row: row['center'].within(poly.geometry[0]), axis=1)
    poly_grid = grid[grid.isin_poly == True]
    poly_grid.crs = {'init': 'epsg:4326', 'no_defs': True}

    return poly_grid

def amenity_in_polygon(amenity_points,poly):
    # returns the amenities that are inside the given polygon
    # When there are zero amenities within the interrogation region, the function returns an empty dataframe as
    # as expected, but also prints out a lot of errors. not a huge issue but annoying.
    # Maybe implement a test for if empty, return 0
    # Example use:
    #         amenity_in_polygon(food_amenities,city_grid.geometry.iloc[38])
    
    return amenity_points[amenity_points.apply(lambda row: row['geometry'].within(poly), axis=1)]

In [4]:
# Define region and radius of interest
place = 'Glasgow City, Scotland, UK' # Glasgow: [55.8642,-4.2518]
# latlon = [55.8642,-4.2518]
# dist = 3*1000 # distance in km

# Get map network for bike only
# graph = ox.graph_from_point(latlon, distance=dist, network_type='bike')
graph = ox.graph_from_place(place, network_type='bike')

# Get intersection and streets from graph
nodes, streets = ox.graph_to_gdfs(graph)

In [5]:
# Generate city grid for interrogation
city = ox.gdf_from_place(place)
city['center_x'] = city['geometry'].centroid.x
city['center_y'] = city['geometry'].centroid.y
city_grid = gridify_polygon(city,0.01)
city_grid.geometry

17     POLYGON ((-4.38320 55.92128, -4.37320 55.92128...
18     POLYGON ((-4.38320 55.91128, -4.37320 55.91128...
19     POLYGON ((-4.38320 55.90128, -4.37320 55.90128...
20     POLYGON ((-4.38320 55.89128, -4.37320 55.89128...
23     POLYGON ((-4.38320 55.86128, -4.37320 55.86128...
                             ...                        
486    POLYGON ((-4.09320 55.87128, -4.08320 55.87128...
487    POLYGON ((-4.09320 55.86128, -4.08320 55.86128...
488    POLYGON ((-4.09320 55.85128, -4.08320 55.85128...
502    POLYGON ((-4.08320 55.87128, -4.07320 55.87128...
504    POLYGON ((-4.08320 55.85128, -4.07320 55.85128...
Name: geometry, Length: 254, dtype: geometry

In [15]:
# get amentities for place
bike_features = ["bicycle_parking",'bicycle_repair_station']
bike_amenities = ox.pois_from_place(place, amenities=bike_features)

food_features = ['cafe']
food_amenities = ox.pois_from_place(place, amenities=food_features)
# Convert all amenities to points if they are polygons
food_amenities['geometry'] = food_amenities.apply(lambda row: row['geometry'].centroid if type(row['geometry']) == Polygon 
                     else row['geometry'], axis=1)
# food_amenities = food_amenities[food_amenities.geom_type == "Point"]

In [44]:
# Write function to identify distance to nearest points of interest
food_amenities

,osmid,geometry,addr:city,addr:housenumber,addr:postcode,addr:street,amenity,cuisine,diet:vegan,diet:vegetarian,...,old_railway_operator,drive_through,landuse,payment:credit_cards,payment:debit_cards,payment:mastercard,payment:visa,start_date,tourism,Dist
287533975,287533975,POINT (-4.28199 55.87279),Glasgow,42,G12 8PB,Otago Lane,cafe,coffee_shop,yes,only,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.060918
364782285,364782285,POINT (-4.28351 55.86468),NaN,NaN,NaN,NaN,cafe,coffee_shop,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.063164
436488459,436488459,POINT (-4.29547 55.87395),NaN,NaN,NaN,NaN,cafe,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.048215
437381247,437381247,POINT (-4.30536 55.87574),NaN,NaN,NaN,NaN,cafe,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.038737
437381252,437381252,POINT (-4.29331 55.87534),NaN,NaN,NaN,NaN,cafe,coffee_shop,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.049529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426154590,426154590,POINT (-4.27288 55.86829),NaN,NaN,NaN,NaN,cafe,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.071063
479586799,479586799,POINT (-4.26404 55.86647),NaN,NaN,NaN,NaN,cafe,burger,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.079932
566765408,566765408,POINT (-4.34104 55.80401),NaN,NaN,G53 7RH,Leggatston Drive,cafe,coffee_shop,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.092308
598305844,598305844,POINT (-4.37899 55.90347),Glasgow,NaN,G15 6RX,Duntreath Avenue,cafe,coffee_shop,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.041418


In [61]:
# find distance between point and top n locations


food_amenities.head()


287533975    False
364782285    False
436488459    False
437381247    False
437381252    False
             ...  
426154590    False
479586799    False
566765408    False
598305844    False
724201306    False
Length: 273, dtype: bool

In [47]:
m = folium.Map([city.center_y, city.center_x],
               zoom_start=11,
               tiles="CartoDb dark_matter")

style_city = {'color':'#ebc923 ', 'fillColor': '#ebc923 ', 'weight':'1', 'fillOpacity' : 0.1}
folium.GeoJson(city, style_function=lambda x: style_city).add_to(m)

# grid points
locs = zip(city_grid.center_y, city_grid.center_x)
for location in locs:
    folium.CircleMarker(location=location, 
        color = "white",   radius=1).add_to(m)

style_region = {'color':'#1FFD09 ', 'fillColor': '#1FFD09 ', 'weight':'1', 'fillOpacity' : 0.1}
folium.GeoJson(city_grid.geometry.iloc[39], style_function=lambda x: style_region).add_to(m)
# plot bikable streets
# m = folium.Map(latlon,
#                zoom_start=15,
#                tiles="CartoDb dark_matter")
# folium.GeoJson(streets, style_function=lambda x: style).add_to(m)

# add cafes
locs = zip(food_amenities.geometry.y, food_amenities.geometry.x)
for location in locs:
    folium.CircleMarker(location=location, 
        color = "red",   radius=1).add_to(m)



m.save("city_view.html")
m